In [4]:
import chardet

# Detectar la codificación
with open('informacion.csv', 'rb') as f:
    result = chardet.detect(f.read())
    encoding = result['encoding']

In [5]:
import pandas as pd

# Intentar cargar el archivo CSV con la codificación detectada y el delimitador correcto
try:
    df = pd.read_csv('informacion.csv', encoding=encoding, sep=';', on_bad_lines='skip')
    # Asumiendo que el CSV tiene columnas 'pregunta' y 'respuesta'
    preguntas = df['pregunta'].tolist()
    respuestas = df['respuesta'].tolist()
except pd.errors.ParserError as e:
    print(f"Error al parsear el archivo CSV: {e}")

In [6]:
preguntas

['¿QUÉ ES LA VIOLENCIA INTRAFAMILIAR?',
 '¿CON QUÉ OTROS NOMBRES ES CONOCIDA LA VIOLENCIA INTRAFAMILIAR?',
 '¿CUÁL ES EL OBJETIVO DE LA VIOLENCIA INTRAFAMILIAR?',
 '¿CUÁLES SON LOS EFECTOS DE LA VIOLENCIA INTRAFAMILIAR PARA LA VÍCTIMA?',
 '¿CUÁLES SON LOS EFECTOS DE LA VIOLENCIA INTRAFAMILIAR PARA LA SOCIEDAD?',
 '¿CUÁLES SON LAS CAUSAS DE LA VIOLENCIA INTRAFAMILIAR?',
 '¿CUÁLES SON LOS TIPOS DE VIOLENCIA INTRAFAMILIAR SEGÚN SU NATURALEZA?',
 '¿EN QUÉ CONSISTE LA VIOLENCIA FÍSICA?',
 '¿EN QUÉ CONSISTE LA VIOLENCIA PSICOLÓGICA O EMOCIONAL?',
 '¿EN QUÉ CONSISTE LA VIOLENCIA SEXUAL?',
 '¿EN QUÉ CONSISTE LA VIOLENCIA ECONÓMICA?',
 '¿CUÁLES SON FACTORES QUE RESPALDAN EL DESARROLLO DE LA VIOLENCIA INTRAFAMILIAR?',
 '¿CÓMO SE DESARROLLAN LOS FACTORES QUE RESPALDAN EL DESARROLLO DE LA VIOLENCIA INTRAFAMILIAR?',
 '¿CUÁLES SON LAS PRINCIPALES SECUELAS A NIVEL FÍSICO DE UNA VÍCTIMA DE VIOLENCIA INTRAFAMILIAR?',
 '¿CUÁLES SON LAS PRINCIPALES SECUELAS A NIVEL PSICOLÓGICO DE UNA VÍCTIMA DE VIOLENCIA

In [7]:
respuestas

['Es el conjunto de actitudes o comportamientos de abuso (ser sexual, físico o psicológico) perpetrados en el hogar o unidad doméstica, generalmente por un miembro de la familia que vive con la víctima, que puede ser esta varón o mujer, infante, adolescente o adulto, con el empleo deliberado de la fuerza. Esto ocurre entre padres, hijos y padres y entre hermanos, por ejemplo. Además, es considera como un grave obstáculo para el desarrollo y la paz.',
 'La violencia intrafamiliar también es conocida como violencia doméstica, violencia familiar o violencia en las familias.',
 'El objetivo de la violencia intrafamiliar es vencer la resistencia de la víctima y obtener su subyugación, es decir, controlarla y dominarla.',
 'Perjudican su integridad corporal y psicológica. Repercuten sobre su independencia e individualidad. Miedo, pánico o angustia. Heridas y lesiones físicas que, en los peores casos, pueden conducir a la muerte. Altos niveles de ansiedad por miedo a volver a ser violentados.

In [11]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "t5-small"  # o el modelo que prefieras
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def prepare_qa_data(preguntas, respuestas):
    inputs = []
    targets = []
    for pregunta, respuesta in zip(preguntas, respuestas):
        input_text = f"Pregunta: {pregunta} Respuesta:"
        target_text = f" {respuesta}"
        
        inputs.append(input_text)
        targets.append(target_text)
    
    model_inputs = tokenizer(inputs, padding=True, truncation=True, return_tensors="pt")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, padding=True, truncation=True, return_tensors="pt")
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

qa_dataset = prepare_qa_data(preguntas, respuestas)

/Users/carolinasv/Documents/VS_Code/ALMA/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [14]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    predict_with_generate=True,
    fp16=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=qa_dataset,
    tokenizer=tokenizer,
)

/Users/carolinasv/Documents/VS_Code/ALMA/venv/lib/python3.12/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
trainer.train()

  0%|          | 0/3 [00:00<?, ?it/s]

AttributeError: 'list' object has no attribute 'keys'